# `Individual` models

In [1]:
# Sconda conda activate tensorflow2; cd /mnt/BioAdHoc/Groups/RaoLab/Edahi/Projects/09.TimeCourse5hmC/ForFerhatGit/FCDNN/results/deeplift_analysis/; python

In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score,
)

import os

import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
)
from tensorflow.keras.initializers import GlorotUniform

from tensorflow.keras.optimizers import Adam
from keras.models import model_from_json
from keras import backend as K

import sys
import pkg_resources

/scratch/ipykernel_3768460/193332328.py:28: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## Declare some constants

In [3]:
out_dir = "/mnt/BioAdHoc/Groups/RaoLab/Edahi/Projects/09.TimeCourse5hmC/ForFerhatGit/FCDNN/results/fcdnn_keras_hyperparams_table"
keras_model_json = "./models/clean_samples_model.json"
keras_model_weights = "./models/clean_samples_weights.h5"
OutputLayer = "sigmoid"
data_dir = "/mnt/BioAdHoc/Groups/RaoLab/Edahi/Projects/09.TimeCourse5hmC/ForFerhatGit/FCDNN/data/C2B"
sample_names = pd.read_csv(
    "/mnt/BioAdHoc/Groups/RaoLab/Edahi/Projects/09.TimeCourse5hmC/ForFerhatGit/FCDNN/filtered_files_surnames.csv",
    header=None,
)[7].tolist()

### Loading Datasets

In [4]:
def load_dataset(data_dir, sample, print_shape: bool = False):
    preffix = os.path.join(data_dir, sample)
    X = np.load(preffix + "_Train_X.npz")["arr_0"].T
    Y = np.load(preffix + "_Train_Y.npz")["arr_0"].T
    X_dev = np.load(preffix + "_Dev_X.npz")["arr_0"].T
    Y_dev = np.load(preffix + "_Dev_Y.npz")["arr_0"].T
    X_test = np.load(preffix + "_Test_X.npz")["arr_0"].T
    Y_test = np.load(preffix + "_Test_Y.npz")["arr_0"].T
    if print_shape:
        print(
            f"""
X_train shape: {X.shape}
Y_train shape: {Y.shape}

X_dev shape: {X_dev.shape}
Y_dev shape: {Y_dev.shape}

X_test shape: {X_test.shape}
Y_test shape: {Y_test.shape}
"""
        )
    return X, Y, X_dev, Y_dev, X_test, Y_test

## Sequential-style FCDNN

### Function to remove layer that is causing issues:

In [5]:
# Load the original JSON data
def json_remove_input_layer(json_file):
    # Read json
    with open(json_file, "r") as file:
        data = json.load(file)
        file.close()
    # Check if it has the problem and delete
    if (
        data["config"]["layers"]
        and data["config"]["layers"][0]["class_name"] == "InputLayer"
    ):
        del data["config"]["layers"][0]
        print("deleted 'InputLayer' layer config")
        # Save the modified data back to a new JSON file
        with open(json_file, "w") as file:
            json.dump(data, file, indent=4)
            file.close()
    else:
        print("Model does not contain 'InputLayer' in layer config")


# Remove the first layer from the 'layers' list within the 'config' key

### Function to make Keras model

In [13]:
# Helper function to identify layers structure:
# I need to know how many layers to set properly the in and out and the intermediate ones
def id_hidden_layers(layers: list):
    n_hidden = None
    hidden_layers = False
    n_layers = len(layers)
    if n_layers >= 3:
        hidden_layers = True
        n_hidden = n_layers - 2
    return hidden_layers, n_hidden

In [16]:
# Make this a function to get a cleaner code:
def make_keras_model(X, layers: list):
    keras_model = Sequential()
    hidden_layers, n_hidden = id_hidden_layers(layers)
    layer_name = 1
    # Input layer
    keras_model.add(
        Dense(
            int(layers[0]),
            activation="relu",
            name=str(layer_name),
            input_dim=X.shape[1],
            kernel_initializer=GlorotUniform(),
        )
    )
    keras_model.add(Dropout(0.15))
    layer_name += 1

    # Hidden Layer(s)
    if hidden_layers:
        for l in range(n_hidden):
            keras_model.add(
                Dense(
                    int(layers[l + 1]),
                    activation="relu",
                    name=str(layer_name),
                    kernel_initializer=GlorotUniform(),
                )
            )
            keras_model.add(Dropout(0.15))
            layer_name += 1

    # Output layer
    keras_model.add(
        Dense(
            int(layers[-1]),
            activation="relu",
            name=str(layer_name),
            kernel_initializer=GlorotUniform(),
        )
    )
    keras_model.add(
        Dense(
            1, activation="sigmoid", name="Output", kernel_initializer=GlorotUniform()
        )
    )
    keras_model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return keras_model

### Function to test model

In [26]:
def test_and_auc(
    keras_model,
    X_test,
    Y_test,
):

    ProbableValues = keras_model.predict(X_test)
    Obs = pd.Series(Y_test.flatten().astype(int))
    Prob = pd.Series(ProbableValues.flatten().astype(float))
    auc_score = round(roc_auc_score(Obs, Prob), 4)

    return auc_score

### Declare layers and results dict


In [29]:
layers_set = [
    [200, 100, 50],
    [100, 100, 100],
    [50, 50, 50],
    [200, 200],
    [100, 100],
    [50, 50],
]

auc_results = dict()
for i in range(len(layers_set)):
    auc_results[i] = []

### Train Models

In [30]:
for i, sample in enumerate(sample_names):
    # load data:
    print(sample)
    X, Y, X_dev, Y_dev, X_test, Y_test = load_dataset(data_dir, sample)
    # Iterate over the layers:
    for l in range(len(layers_set)):
        print(l)
        layers = layers_set[l]
        # clean session
        K.clear_session()
        # define the keras model
        sample_model = make_keras_model(X, layers)
        sample_model.fit(x=X, y=Y, batch_size=1024, epochs=40, verbose=0)
        auc_results[l].append(test_and_auc(sample_model, X_test, Y_test))

mm10_Bcell_Activated_24_Rep1
0
1
2
3
4
5
mm10_Bcell_Activated_48_Rep1
0
1
2
3
4
5
mm10_Bcell_Activated_72_Rep1
0
1
2
3
4
5
mm10_Bcell_Resting_Rep1
0
1
2
3
4
5
mm10_Bergmann_Glia_Rep2
0
1
2
3
4
5
mm10_CD4_Naive_Tcell_Rep2
0
1
2
3
4
5
mm10_CD4_SinglePositive_Tcell_Rep1
0
1
2
3
4
5
mm10_CD8_Naive_Tcell_Rep2
0
1
2
3
4
5
mm10_CD8_SinglePositive_Tcell_Rep2
0
1
2
3
4
5
mm10_Cardiomyocites_Adult_Rep1
0
1
2
3
4
5
mm10_Cardiomyocites_TAC_Rep2
0
1
2
3
4
5
mm10_Colon_Epithelia_Rep2
0
1
2
3
4
5
mm10_DoublePositive_Tcell_Rep2
0
1
2
3
4
5
mm10_E13_Frontal_Cortex_Rep1
0
1
2
3
4
5
mm10_E14_mESC_Rep3
0
1
2
3
4
5
mm10_Female_Cerebellum_Rep1
0
1
2
3
4
5
mm10_Female_Cortex_Rep1
0
1
2
3
4
5
mm10_Female_Hippocampus_Rep1
0
1
2
3
4
5
mm10_Female_Hypothalamus_Rep1
0
1
2
3
4
5
mm10_Female_Thalamus_Rep1
0
1
2
3
4
5
mm10_Granule_Cells_Rep1
0
1
2
3
4
5
mm10_Hematopo_CommonMyeloydProgenitor_Rep1
0
1
2
3
4
5
mm10_Hematopo_Granulocyte_Monocyte_Progenitor_Rep1
0
1
2
3
4
5
mm10_Hematopo_LSK_Rep1
0
1
2
3
4
5
mm10_Hematop

In [38]:
column_names = []
for l in range(len(layers_set)):
    column_names.append("_".join(map(str, layers_set[l])))

In [71]:
auc_per_layer_conf = pd.DataFrame(auc_results)
auc_per_layer_conf.columns = column_names
auc_per_layer_conf.index = sample_names
auc_per_layer_conf

,200_100_50,100_100_100,50_50_50,200_200,100_100,50_50
mm10_Bcell_Activated_24_Rep1,0.8994,0.9027,0.8828,0.8978,0.8884,0.8853
mm10_Bcell_Activated_48_Rep1,0.9047,0.9063,0.8978,0.9065,0.8904,0.8872
mm10_Bcell_Activated_72_Rep1,0.8896,0.8858,0.8782,0.8881,0.8874,0.8801
mm10_Bcell_Resting_Rep1,0.9076,0.9003,0.8969,0.9012,0.9105,0.8866
mm10_Bergmann_Glia_Rep2,0.8328,0.8208,0.8182,0.8363,0.8289,0.7983
mm10_CD4_Naive_Tcell_Rep2,0.9298,0.9318,0.9277,0.9266,0.9224,0.9155
mm10_CD4_SinglePositive_Tcell_Rep1,0.9095,0.8992,0.8912,0.9066,0.9019,0.8948
mm10_CD8_Naive_Tcell_Rep2,0.9321,0.9286,0.9325,0.9333,0.9239,0.9205
mm10_CD8_SinglePositive_Tcell_Rep2,0.9293,0.9253,0.9166,0.9303,0.9229,0.9099
mm10_Cardiomyocites_Adult_Rep1,0.7955,0.7926,0.7872,0.7986,0.7786,0.7556


In [60]:
summary_auc = []
for l in auc_per_layer_conf.columns:
    summary_auc.append(
        auc_per_layer_conf[l].describe()[[3, 4, 5, 1, 6, 7]].values.round(4).tolist()
    )
summary_auc

[[0.7382, 0.8383, 0.8547, 0.8593, 0.8864, 0.9321],
 [0.7253, 0.8387, 0.8534, 0.856, 0.8858, 0.9318],
 [0.6893, 0.8182, 0.8393, 0.843, 0.8782, 0.9325],
 [0.7358, 0.8344, 0.8506, 0.8569, 0.8875, 0.9333],
 [0.7255, 0.8277, 0.8469, 0.8505, 0.8874, 0.9283],
 [0.6855, 0.822, 0.8353, 0.8373, 0.8762, 0.9205]]

In [69]:
summary_auc_per_layer = pd.DataFrame(summary_auc)
summary_auc_per_layer.columns = (
    auc_per_layer_conf[l].describe()[[3, 4, 5, 1, 6, 7]].index.tolist()
)
summary_auc_per_layer.index = auc_per_layer_conf.columns
summary_auc_per_layer

,min,25%,50%,mean,75%,max
200_100_50,0.7382,0.8383,0.8547,0.8593,0.8864,0.9321
100_100_100,0.7253,0.8387,0.8534,0.8560,0.8858,0.9318
50_50_50,0.6893,0.8182,0.8393,0.8430,0.8782,0.9325
200_200,0.7358,0.8344,0.8506,0.8569,0.8875,0.9333
100_100,0.7255,0.8277,0.8469,0.8505,0.8874,0.9283
50_50,0.6855,0.8220,0.8353,0.8373,0.8762,0.9205


In [72]:
auc_per_layer_conf.to_csv("./distribution_of_auc_per_layer_config.csv")

In [73]:
summary_auc_per_layer.to_csv("./summary_of_auc_per_layer_config.csv")